In [1]:
import pandas as pd

In [3]:
# Chargement du classement depuis le fichier CSV
data = pd.read_csv('data/algeria/d1-2024-2025.csv')

In [4]:
# Rename columns for easier handling
data.columns = ['Pos', 'Team', 'Pts', 'G', 'N', 'P', 'Bp', 'Bc', 'diff', 'J']
data.head()

,Pos,Team,Pts,G,N,P,Bp,Bc,diff,J
0,1,MC Alger,41,11,8,2,27,15,12,21
1,2,CR Belouizdad,40,11,7,4,31,13,18,22
2,3,JS Kabylie,37,10,7,5,29,21,8,22
3,4,ES Setif,34,9,7,6,18,15,3,22
4,5,USM Alger,33,8,9,3,19,10,9,20


In [5]:
# League averages
league_avg_goals_for = data['Bp'].sum() / data['J'].sum()
league_avg_goals_against = data['Bc'].sum() / data['J'].sum()

print(f"League Avg Goals Scored per Match: {league_avg_goals_for:.2f}")

League Avg Goals Scored per Match: 0.93


In [6]:
# Calculate team attack and defense strength
data['Attack_Strength'] = (data['Bp'] / data['J']) / league_avg_goals_for
data['Defense_Weakness'] = (data['Bc'] / data['J']) / league_avg_goals_for

data[['Team', 'Attack_Strength', 'Defense_Weakness']].head()

,Team,Attack_Strength,Defense_Weakness
0,MC Alger,1.388889,0.771605
1,CR Belouizdad,1.522166,0.638328
2,JS Kabylie,1.423962,1.031145
3,ES Setif,0.883838,0.736532
4,USM Alger,1.026235,0.540123


In [10]:
def predict_match(team1, team2, data, max_goals=5, print_output=True):
    import numpy as np
    from scipy.stats import poisson

    # League average goals (you can also pass this in if you want to optimize)
    league_avg_goals = data['Bp'].sum() / data['J'].sum()

    # Get team stats
    t1 = data[data['Team'] == team1].iloc[0]
    t2 = data[data['Team'] == team2].iloc[0]

    # Expected goals using Poisson assumption
    exp_g1 = t1['Attack_Strength'] * t2['Defense_Weakness'] * league_avg_goals
    exp_g2 = t2['Attack_Strength'] * t1['Defense_Weakness'] * league_avg_goals

    # Poisson probability matrix
    prob_matrix = np.outer(
        [poisson.pmf(i, exp_g1) for i in range(max_goals + 1)],
        [poisson.pmf(j, exp_g2) for j in range(max_goals + 1)]
    )

    # Match outcome probabilities
    home_win_prob = np.tril(prob_matrix, -1).sum()
    draw_prob = np.trace(prob_matrix)
    away_win_prob = np.triu(prob_matrix, 1).sum()

    # Decimal odds (implied, without margin)
    home_odds = round(1 / home_win_prob, 2)
    draw_odds = round(1 / draw_prob, 2)
    away_odds = round(1 / away_win_prob, 2)

    if print_output:
        print(f"\n📊 Expected Goals:")
        print(f"{team1}: {exp_g1:.2f}, {team2}: {exp_g2:.2f}")

        print(f"\n📈 Match Outcome Probabilities:")
        print(f"{team1} Win: {home_win_prob:.2%}")
        print(f"Draw: {draw_prob:.2%}")
        print(f"{team2} Win: {away_win_prob:.2%}")

        print(f"\n🎯 Implied Decimal Odds:")
        print(f"{team1} Win: {home_odds}")
        print(f"Draw: {draw_odds}")
        print(f"{team2} Win: {away_odds}")

    return {
        'expected_goals': {team1: exp_g1, team2: exp_g2},
        'probabilities': {
            f'{team1}_win': home_win_prob,
            'draw': draw_prob,
            f'{team2}_win': away_win_prob,
        },
        'decimal_odds': {
            f'{team1}_win': home_odds,
            'draw': draw_odds,
            f'{team2}_win': away_odds,
        },
        'probability_matrix': prob_matrix
    }

In [11]:
predict_match('Biskra', 'MC Alger', data)


📊 Expected Goals:
Biskra: 0.35, MC Alger: 1.14

📈 Match Outcome Probabilities:
Biskra Win: 11.40%
Draw: 32.55%
MC Alger Win: 55.94%

🎯 Implied Decimal Odds:
Biskra Win: 8.78
Draw: 3.07
MC Alger Win: 1.79


{'expected_goals': {'Biskra': np.float64(0.350729517396184),
  'MC Alger': np.float64(1.1363636363636365)},
 'probabilities': {'Biskra_win': np.float64(0.11395859131614253),
  'draw': np.float64(0.3254974557358513),
  'MC Alger_win': np.float64(0.5594009518989039)},
 'decimal_odds': {'Biskra_win': np.float64(8.78),
  'draw': np.float64(3.07),
  'MC Alger_win': np.float64(1.79)},
 'probability_matrix': array([[2.26028732e-01, 2.56850832e-01, 1.45937973e-01, 5.52795352e-02,
         1.57044134e-02, 3.56918486e-03],
        [7.92749482e-02, 9.00851684e-02, 5.11847548e-02, 1.93881647e-02,
         5.50800133e-03, 1.25181848e-03],
        [1.39020322e-02, 1.57977638e-02, 8.97600217e-03, 3.40000082e-03,
         9.65909324e-04, 2.19524846e-04],
        [1.62528434e-03, 1.84691403e-03, 1.04938297e-03, 3.97493549e-04,
         1.12924304e-04, 2.56646145e-05],
        [1.42508798e-04, 1.61941816e-04, 9.20123956e-05, 3.48531802e-05,
         9.90147164e-06, 2.25033446e-06],
        [9.99640841e-